In [6]:
import urllib.request

In [7]:
urllib.request.urlretrieve("https://archive.ics.uci.edu/dataset/53/iris.zip","data.zip")


('data.zip', <http.client.HTTPMessage at 0x7f4f957586d0>)

In [8]:
!mkdir data
!unzip data.zip -d data/

mkdir: cannot create directory ‘data’: File exists
Archive:  data.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data.zip or
        data.zip.zip, and cannot find data.zip.ZIP, period.


In [9]:
import pandas as pd


In [27]:
df = pd.read_csv("data/iris.data",header=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       150 non-null    float64
 1   1       150 non-null    float64
 2   2       150 non-null    float64
 3   3       150 non-null    float64
 4   4       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [28]:
df

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [29]:
df[4].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [30]:
df[4] = df[4].replace("Iris-setosa",0)
df[4] = df[4].replace("Iris-versicolor",1)
df[4] = df[4].replace("Iris-virginica",2)


/tmp/ipykernel_11975/322292279.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[4] = df[4].replace("Iris-virginica",2)


In [31]:
df

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [32]:
df[4].unique()

array([0, 1, 2])

In [33]:
df = df.sample(frac = 1).reset_index(drop=True)
df

,0,1,2,3,4
0,5.6,3.0,4.5,1.5,1
1,5.0,3.5,1.6,0.6,0
2,5.8,2.7,3.9,1.2,1
3,6.4,2.9,4.3,1.3,1
4,6.2,2.9,4.3,1.3,1
...,...,...,...,...,...
145,7.1,3.0,5.9,2.1,2
146,6.3,3.3,6.0,2.5,2
147,5.2,4.1,1.5,0.1,0
148,7.4,2.8,6.1,1.9,2


In [34]:
df = df[[4,0,1,2,3]]
df

,4,0,1,2,3
0,1,5.6,3.0,4.5,1.5
1,0,5.0,3.5,1.6,0.6
2,1,5.8,2.7,3.9,1.2
3,1,6.4,2.9,4.3,1.3
4,1,6.2,2.9,4.3,1.3
...,...,...,...,...,...
145,2,7.1,3.0,5.9,2.1
146,2,6.3,3.3,6.0,2.5
147,0,5.2,4.1,1.5,0.1
148,2,7.4,2.8,6.1,1.9


In [35]:
train_df = df[:120]
val_df = df[120:]

In [36]:
train_df.shape,val_df.shape

((120, 5), (30, 5))

In [37]:
import boto3


In [38]:
bucket_name = "sagemaker-ajeet-s3-bucket-test-1"


In [39]:
train_df.to_csv("data.csv",header = False, index  = False)



In [40]:
key = 'data/train/data'
url = 's3://{}/{}/'.format(bucket_name ,key)
boto3.Session().resource("s3").Bucket(bucket_name).Object(key).upload_file('data.csv')


In [41]:
val_df.to_csv("data.csv",header = False, index  = False)
key = 'data/val/data'
url = 's3://{}/{}/'.format(bucket_name ,key)
boto3.Session().resource("s3").Bucket(bucket_name).Object(key).upload_file('data.csv')

In [ ]:
# create model

In [59]:
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

In [60]:
key = 'model/xgb_model'
s3_output_location = url = 's3://{}/{}'.format(bucket_name,key)


In [61]:
xgb_model = sagemaker.estimator.Estimator(
     get_image_uri(boto3.Session().region_name,'xgboost'),
     get_execution_role(),
     train_instance_count = 1,
     train_instance_type = 'ml.m4.xlarge',
     train_volume_size = 5,
     output_path = s3_output_location,
     sagemaker_session = sagemaker.Session()
)
    
    

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [64]:
xgb_model.set_hyperparameters(max_depth = 5,
                              eta = 0.2,
                              gamma = 4,
                              min_child_weight = 6,
                              silent = 0,
                              objective = 'multi:softmax',
                              num_class = 3,
                              num_round = 10
                             )

In [ ]:
# train model
# load data from s3 bucket

In [52]:
train_data = "s3://{}/{}/".format(bucket_name ,'data/train')
val_data = "s3://{}/{}/".format(bucket_name ,'data/val')

In [54]:
train_channel = sagemaker.session.s3_input(train_data,content_type = 'text/csv')
val_channel = sagemaker.session.s3_input(val_data,content_type = 'text/csv')


The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [55]:
data_channel = {'train': train_channel,'validation':val_channel}


In [ ]:
# model fit

In [65]:
xgb_model.fit(inputs = data_channel)

INFO:sagemaker:Creating training-job with name: xgboost-2024-07-10-17-14-06-811


2024-07-10 17:14:06 Starting - Starting the training job...
2024-07-10 17:14:26 Starting - Preparing the instances for training...
2024-07-10 17:14:49 Downloading - Downloading input data...
2024-07-10 17:15:19 Downloading - Downloading the training image......
2024-07-10 17:16:30 Training - Training image download completed. Training in progress.
2024-07-10 17:16:30 Uploading - Uploading generated training modelArguments: train
[2024-07-10:17:16:25:INFO] Running standalone xgboost training.
[2024-07-10:17:16:25:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8460.11mb
[2024-07-10:17:16:25:INFO] Determined delimiter of CSV input is ','
[17:16:25] S3DistributionType set as FullyReplicated
[17:16:25] 120x4 matrix with 480 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-07-10:17:16:25:INFO] Determined delimiter of CSV input is ','
[17:16:25] S3DistributionType set as FullyReplicated
[17:16:25] 30x4 matrix

In [66]:
xgb_predictior = xgb_model.deploy(initial_instance_count = 1,
                                  instance_type = 'ml.m4.xlarge')


INFO:sagemaker:Creating model with name: xgboost-2024-07-10-17-19-38-274
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-07-10-17-19-38-274
INFO:sagemaker:Creating endpoint with name xgboost-2024-07-10-17-19-38-274


-----!

In [ ]:
xgb_predictior